<div align="right"><i>COM418 - Computers and Music</i></div>
<div align="right"><a href="https://people.epfl.ch/paolo.prandoni">Lucie Perrotta</a>, <a href="https://www.epfl.ch/labs/lcav/">LCAV, EPFL</a></div>

<p style="font-size: 30pt; font-weight: bold; color: #B51F1F;">Synthesizers and Oscillators</p>

In [ ]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
from scipy.interpolate import RectBivariateSpline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from IPython.display import Audio

import matplotlib
figsize=(10,5)
linewidth=2.5
matplotlib.rcParams.update({'font.size': 16})

In [ ]:
# Constants
fs = 44100
time = np.linspace(0, 2, 2*fs)
samples = np.arange(2*fs)
fourier = np.linspace(0, .2*np.pi, 2*fs)

A synthesizer is an electronic musical instrument that allows to generate audio signals. It generally presents as a piano keyboard, along with many knobs for tuning parameters, grouped in sections that correspond to the different stages of shaping the sound of the synthesizer. The first stage is to generate a sound wave, and is called an **oscillator**. This can be done in several ways, like by adding sines, or modulating them together. The wave is then sent to the next stages which are filters changing its shape, for instance low frequency oscillators (**LFOs**), equalizers (EQs), etc.

<img src="https://images-na.ssl-images-amazon.com/images/I/71KdQUycyQL._AC_SL1500_.jpg" alt="Drawing" style="width: 35%;"/>

In this notebook, we will implement 4 types of oscillators: the digital, the additive, the FM, and the wavetable oscillators. We will also implement a LFO.

## 1. Additive and digital synthesis

According to Fourier theory, summing up sines allows to create any waveform. An additive oscialltor hence generates waveforms by adding sines with different amplitudes, phases, and angular frequencies together. The created waveform is periodic and can also be defined by an amplitude $A$, an angular frequency $\omega_c$ and a phase $\phi$. 

\begin{align*}
    wave[A, \omega_c, \phi; n]
\end{align*}

An oscillator containing an additive oscillator is also called an additive synthesizer, and typically features 4 basic wavesforms:
- The sine wave
- The square wave
- The sawtooth wave
- The triangle wave

that can each be fully described by the above cited parameters.

With the coming of digital technology, summing sines was no longer needed to generate waveforms as the synthesizer could simply use mathematical functions to draw the shape of the wave.

Here we write down 4 functions for generating each of the 4 basic waveforms. Each function is given in 2 versions, the digital version which simply generates the waveform using any mathematical function at disposal, and the additive version which uses a sum of sines to approximate the waveform. Note that additive implementation of each funtion features an additional parameter "order" controlling the number of sines to use in the sum.

### 1.1. Sine wave

The sine wave is the most fundamental waveform that an oscillator can generate. In an analog oscillator, all waveforms are generated as a (large but) finite sum of sines. It is composed of only one frequency and hence has a rather quiet sound. Its shape being smooth and lacking edges, the sine waveform has a rather natural, colorless sound.

\begin{align*}
    sine[A, \omega_c, \phi; n] = A \sin (\omega_c n - \phi)
\end{align*}

In [ ]:
def sine(A, w, phi, n):
    return A * np.sin(w*n - phi)

In [ ]:
plt.figure(figsize=figsize)
plt.plot(time, sine(1, 5, 0, time), linewidth=linewidth,color="slateblue",  label="Sine")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend(loc='upper right')
#plt.savefig("sine.pdf" ,bbox_inches='tight')
plt.show()

We can subsequently plot the FFT of the sine. Notice the spectral leakage in the plot.

In [ ]:
plt.figure(figsize=figsize)
plt.magnitude_spectrum(sine(1, 500, 0, time), Fs=fs, scale="dB", linewidth=linewidth,color="slateblue",  label="Sine")
plt.xlim(0, 500)
plt.ylim(-200, 0)
plt.legend(loc='upper right')
#plt.savefig("sine_fft.pdf",bbox_inches='tight')
plt.show()

In [ ]:
Audio(sine(1, 0.05, 0, samples), rate=fs)

### 1.2. Square wave

The square wave is simply as signal alternating between $-A$ and $+A$. Its equation can be simply written as

\begin{align*}
    square[A, \omega_c, \phi;n] = A \cdot sgn \left( \sin (\omega_c n -\phi \right) ).
\end{align*}

This implementation is very simple and can be efficiently coded for digital synthesizers. 

In [ ]:
def square(A, w, phi, n):
    return A*np.sign( np.sin( w*n - phi))

The corresponding additive oscialltor equation reads
\begin{align*}
    square[A, \omega_c, \phi;n] = \frac{4A}{\pi} \sum_{k=1}^{\infty} \frac{ \sin ((2k-1)\omega_c n - \phi)}{2k - 1} ,
\end{align*}
where the infinite sum is in practice approximated with a high order.

In [ ]:
def square_sin(A, w, phi, n, order=20):
    out = 0
    for k in range(1, order+1):
        out += np.sin((2*k-1)*w*n - phi)/(2*k-1)
        
    return 4*A/np.pi * out

The square wave has a lot of harmonics that decrease slowly in amplitude. This give the square wave a sharper, brighter sound than the sine wave. The square wave was used a lot in early videogames consoles, such as the Commodore 64, and is now often associated with retro games music, or *chiptune*. In the figure are compared a digital square wave, generated directly using the exact equation described above, as well as its additive synthesis approximation. One can observe in the FFT plot that only the $k$ first harmonics of the square wave are generated.

In [ ]:
def update(order=10):

    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(time, square_sin(1, 5, 0, time, order), linewidth=linewidth, color="orange", label="Additive square")
    ax.plot(time, square(1, 5, 0, time), linewidth=linewidth, color="slateblue", label="Digital square")
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.legend(loc='upper right')
    #plt.savefig("square.pdf",bbox_inches='tight')
    plt.show()
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.magnitude_spectrum(square(1, 50, 0, time), Fs=fs, scale="dB", linewidth=linewidth,color="slateblue",  label="Digital square")
    ax.magnitude_spectrum(square_sin(1, 50, 0, time, order), Fs=fs, scale="dB", linewidth=linewidth, color="orange", label="Additive square")
    plt.legend(loc='upper right')
    plt.ylim(-120, 0)
    plt.xlim(0, 300)
    #plt.savefig("square_fft.pdf",bbox_inches='tight')
    plt.show()

In [ ]:
widgets.interact(update, order=(1, 20, 1));

An issue that is met when generating a square wave using additive synthesis is the **Gibbs phenomenon**. This phenomenon corresponds to the apparition of small vertical spikes at the corners of the square wave. An interactive animation from Dr. Prandoni's introductory DSP class highlights the Gibbs phenomenon for classical waveforms: https://www.sp4comm.org/gibbs/gibbs.html. The phenomenon highlights the fact that Fourier series can describe perfectly any smooth function, but meet limitations with non smooth functions, such as the square wave. Indeed, the additive synthesis square wave does not converge to the digital square wave, even for $k \rightarrow \infty$, and A spike measuring approximately $18\%$ of the amplitude $A$ is always observed for arbitrary great values of $k$. This however does not generate problems from an audio point of view, as the sound of a square wave with spikes is generally considered similar to the sound of a digital square wave. Most additive synthesis synthesizer hence simply use this formula for generating square waves.

In [ ]:
Audio(square(1, 0.05, 0, samples), rate=fs)

### 1.3. Sawtooth wave

The sawtooth wave first increases linearly from $-A$ to $+A$ and then instantly jumps back to $-A$. This is digitally written as

\begin{align*}
    saw[A, \omega_c, \phi;n] = \frac{2A}{\pi} \arctan ( \tan (\omega_c n/2 - \phi) ) .
\end{align*}

In [ ]:
def saw(A, w, phi, n):
    return 2*A/np.pi * np.arctan( np.tan( w*n/2 - phi))

Similarly as for the square wave, analog synthesizer approximate this wave using additive synthesis as

\begin{align*}
    saw[A, \omega_c, \phi;n] = -\frac{2A}{\pi} \sum_{k=1}^{\infty} \frac{(-1)^k}{k}\sin (k \omega_c n - \phi) .
\end{align*}

In [ ]:
def saw_sin(A, w, phi, n, order=20):
    out = 0
    for k in range(1, order+1):
        out += (-1)**k/k * np.sin( k*w*n - phi )
        
    return - 2*A/np.pi * out

One can remark that the Gibbs phenomenon is present as for the square wave, since the sawtooth contains a vertical discontinuity.  The sawtooth wave has a very crispy, aggressive sound and is for that reason quite popular in Trance and House electronic music.

In [ ]:

def update(order=10):

    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(time, saw_sin(1, 5, 0, time, order), linewidth=linewidth, color="orange", label="Additive saw")
    ax.plot(time, saw(1, 5, 0, time), linewidth=linewidth, color="slateblue", label="Digital saw")
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.legend(loc='upper right')
    #plt.savefig("saw.pdf",bbox_inches='tight')
    plt.show()
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.magnitude_spectrum(saw(1, 50, 0, time), Fs=fs, scale="dB", linewidth=linewidth,color="slateblue",  label="Digital saw")
    ax.magnitude_spectrum(saw_sin(1, 50, 0, time, order), Fs=fs, scale="dB", linewidth=linewidth, color="orange", label="Additive saw")
    plt.legend(loc='upper right')
    plt.xlim(0, 300)
    plt.ylim(-120, 0)
    #plt.savefig("saw_fft.pdf",bbox_inches='tight')
    plt.show()

In [ ]:
widgets.interact(update, order=(1, 20, 1));

In [ ]:
Audio(saw(1, 0.05, 0, samples), rate=fs)

### 1.4. Triangle wave

The triangle wave is an alternatively increasing then decreasing linear signal between $-A$ and $+A$. It is digitally written as

\begin{align*}
    tri[A, \omega_c, \phi ;n] = \frac{2A}{\pi} \arcsin ( \sin (\omega_c n - \phi) ).
\end{align*}

In [ ]:
def triangle(A, w, phi, n):
    return 2*A/np.pi * np.arcsin( np.sin( w*n - phi))

The corresponding additive synthesis implementation reads

\begin{align*}
    tri[A, \omega_c, \phi;n] = \frac{8A}{\pi^2} \sum_{k=0}^\infty \frac{ (-1)^k }{ (2k + 1)^2} \sin ((2k + 1) \omega_c n - \phi).
\end{align*}

In [ ]:
def triangle_sin(A, w, phi, n, order=20):
    out = 0
    for k in range(order):
        out += (-1)**k/(2*k+1)**2 * np.sin( (2*k+1)* w*n - phi )
        
    return 8*A/np.pi**2 * out

The triangle wave having no vertical jump, the Gibbs phenomenon is not significant. The additive approximation converges faster than the sawtooth and the square wave towards the digital implementation. One can notice that with $k=2$, the approximation is already quite close to the digital wave. The triangle wave has a sound in between the smoothness of sine and the brightness of a sawtooth. 

In [ ]:

def update(order=2):

    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(time, triangle_sin(1, 5, 0, time, order), linewidth=linewidth, color="orange", label="Additive triangle")
    ax.plot(time, triangle(1, 5, 0, time), linewidth=linewidth, color="slateblue", label="Digital triangle")
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.legend(loc='upper right')
    #plt.savefig("triangle.pdf",bbox_inches='tight')
    plt.show()
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.magnitude_spectrum(triangle(1, 50, 0, time), Fs=fs, scale="dB", linewidth=linewidth,color="slateblue",  label="Digital triangle")
    ax.magnitude_spectrum(triangle_sin(1, 50, 0, time, order), Fs=fs, scale="dB", linewidth=linewidth, color="orange", label="Additive triangle")
    plt.xlim(0, 300)
    plt.ylim(-150, 0)
    plt.legend(loc='upper right')
    #plt.savefig("triangle_fft.pdf",bbox_inches='tight')
    plt.show()

In [ ]:
widgets.interact(update, order=(1, 20, 1));

In [ ]:
Audio(triangle(1, 0.05, 0, samples), rate=fs)

## 2. FM synthesis

Alternatively to the additive synthesis, another sound creation scheme is FM synthesis. FM synthesis relies on Frequency Modulation (FM) as used in radio broadcast. An oscillator produces a waveform, called **the carrier**, that is then modulated with another waveform, called **the modulator**. The carrier is generally a sine wave, while the modulator can have any shape. In FM broadcasting, the carrier has a much higher frequency than the modulator. In FM synthesis, both the carrier and modulator signals have similar, audible frequencies. The carrier wave determines the pitch of the produced note, and the modulator determines its timbre. The equation of modulating a sine carrier with amplitude $A_C$, angular frequency $\omega_C$ and zero phase with an arbitrary modulator wave $x[n]$ reads

\begin{align*}
    y[n] = A_C \cdot \sin \left( \omega_C n + 2 \pi K_f \cdot x[n] \right)
\end{align*}

where $K_f$ is the modulation index, the sensitivity of the frequency modulator.

The output of the modulation can be used as the input of a subsequent modulation (or itself!), and different dispositions of modulator networks can generate different sounds. Modulations can be done in series and parallel and each output can be connected to any input. The behaviour of a modulation network is quite hard to predict, and generally the user experiments with different networks and parameters until they find a satisfying sound.

In [ ]:
def modulator(cw, cA, mod_signal, mod_index=1):
    """
    Block modulating a sine carrier with a modulator.
    cw: the angular frequency of the sine carrier
    cA: the amplitude of the sine carrier
    mod_signal: the arbitrary modulator signal
    mod_index: the modulation index Kf
    """
    return np.array([cA * np.sin(cw*i + mod_index*2.*np.pi*mod_signal[i]) for i in range(mod_signal.size)])

We can now try to modulate several sine signals together and observe how complex the waveform becomes after only a couple iterations!

In [ ]:
sine1 = sine(A=1, w=0.06, phi=-1, n=np.arange(2*fs))

In [ ]:
plt.figure(figsize=figsize)
plt.plot(sine1[:600])
plt.xlabel("Time [samples]")
plt.ylabel("Amplitude")
plt.show()

plt.figure(figsize=figsize)
plt.magnitude_spectrum(sine1, Fs=fs, scale="dB")
plt.ylim(-200,0)
plt.xlim(0, 5000)
plt.show()

In [ ]:

Audio(sine1, rate=fs)

In [ ]:

def update(frequency=.059, modulation=.19):

    sine2 = modulator(cw=frequency, cA=1, mod_signal=sine1, mod_index=modulation)

    plt.figure(figsize=figsize)
    plt.plot(sine2[:600])
    plt.xlabel("Time [samples]")
    plt.ylabel("Amplitude")
    plt.show()

    plt.figure(figsize=figsize)
    plt.magnitude_spectrum(sine2, Fs=fs, scale="dB")
    plt.ylim(-200,0)
    plt.xlim(0, 5000)
    plt.show()

In [ ]:
widgets.interact(update, order=(1, 20, 1));

In [ ]:
sine2 = modulator(cw=0.0599, cA=1, mod_signal=sine1, mod_index=0.29)
Audio(sine2, rate=fs)

In [ ]:
@widgets.interact(frequency=(.00019, .01, .001), modulation=(0, 2, .05))
def update(frequency=.001, modulation=.19):

    sine3 = modulator(cw=frequency, cA=1, mod_signal=sine1, mod_index=modulation)

    plt.figure(figsize=figsize)
    plt.plot(sine3[:600])
    plt.xlabel("Time [samples]")
    plt.ylabel("Amplitude")
    plt.show()

    plt.figure(figsize=figsize)
    plt.magnitude_spectrum(sine3, Fs=fs, scale="dB")
    plt.ylim(-200,0)
    plt.xlim(0, 5000)
    plt.show()

In [ ]:
sine3 = modulator(.00071, 1, sine2, mod_index=.81)
Audio(sine3, rate=fs)

## 3. Working with LFOs

A Low Frequency Oscillator (LFO) is, as the name suggests, an oscillator which operates at a low frequency, generally within $(0, 20]$Hz. LFOs are generally generated using one of the waveforms presented in the above section, and that for all types of synthesizers. For instance, although FM synthesizers do not use waveform oscillators for generating the source sound, they generally still use waveforms for the LFO. Oppositely to a standard oscillator, a LFO does not generate a sound, but is used to automatically control a parameter inside of the synthesizer and to make it change over time. Virtually, any parameter of the amplifier can be controlled by a LFO (even the LFO parameters themselves). For instance, a LFO can be used to automatically control the synthesizer's master volume, to create a tremolo, or to control the phaseof the main oscillator to create a vibrato.

### 3.1. Tremolo

Let's first use a LFO to control the tremolo over a sine wave. We simply use the output of the LFO as the input of the amplitude of the oscillator.

In [ ]:
@widgets.interact(lfo_speed=(.0001, .005, .0001))
def update(lfo_speed=.0015):

    lfo = sine(1, lfo_speed, 0, samples) # Low freqency oscillator
    lfo_tremolo = square(lfo, .06, 0, samples) # LFO'ed sine wave

    fig, ax = plt.subplots(2, figsize=figsize)
    ax[1].plot(lfo[:5000], linestyle="--", alpha=.6)
    ax[0].plot(lfo_tremolo[:5000])
    plt.xlabel("Time [samples]")
    ax[1].set(ylabel='LFO')
    ax[0].set(ylabel='Oscillator')
    ax[0].set(title="LFO tremolo")
    plt.show()

In [ ]:
lfo = sine(1, .0003, 0, samples) # Low freqency oscillator
lfo_tremolo = square(lfo, .06, 0, samples) # LFO'ed sine wave
Audio(lfo_tremolo, rate=fs)

### 3.2. Vibrato

One can also program the phase of an oscialltor with a LFO to create a vibrato.

In [ ]:
@widgets.interact(lfo_amp=(0, 10, 1), lfo_speed=(0.00001, .01, 0.0005))
def update(lfo_amp=5, lfo_speed=0.007):

    lfo = sine(lfo_amp, lfo_speed, 0, samples) # Low freqency oscillator
    lfo_vibrato = square(1, 0.06, lfo, samples) # LFO'ed sine wave

    fig, ax = plt.subplots(2, figsize=figsize)
    ax[1].plot(lfo[:5000], linestyle="--", alpha=.6)
    ax[0].plot(lfo_vibrato[:5000])
    ax[1].set(ylim=(-11,11), ylabel='LFO')
    ax[0].set(ylabel='Oscillator')
    plt.xlabel("Time [samples]")
    ax[0].set(title="LFO vibrato")
    plt.show()

In [ ]:
lfo = sine(5, .0005, 0, samples) # Low freqency oscillator
lfo_vibrato = square(1, 0.06, lfo, samples) # LFO'ed sine wave
Audio(lfo_vibrato, rate=fs)

## 4. Wawetable synthesis

### 4.1. Alternating wavetable

Let's now review a last type of synthesis, **Wavetable synthesis**. Wavetable synthesis was introduced in the late 1970s by Palm Products GmbH but has mostly gained popularity recently due to its easiness of implementation and use on a computer as a VST plugin. The principle is very simple: the synthesizer keeps in memory a table whose entries contain each a single cycle of a different waveform (preferably all of the same length so that they have the same frequency when played back). The user can then select (or even import) a waveform and play it periodically at different speeds, resulting in different frequencies. A key feature of wavetables is the ability to quickly and automatically switch from a waveform to another, while playing.

In [ ]:
def wavetable_alternate(frequency, waveform1, waveform2):
    """
    Simple wavetable generator without interpolation
    frequency: the frequency or the output wave
    waveform1: the function of the first waveform
    waveform2: the function of the second waveform
    return: a 2 seconds long wave alternating between 2 waveforms (1 cycle each, in turn)
    """
    # Indices for one cycle at double the frequency, so that the 2 cycles together become the frequency
    one_cycle = np.linspace(0, 1/frequency, int(fs/2/frequency)) 

    # Generate 1 cycle of each waveform at double the frequency
    wave1 = waveform1(1, frequency*2*np.pi, 0, one_cycle)
    wave2 = waveform2(1, frequency*2*np.pi, 0, one_cycle)

    # Concatenate them to reach 2 seconds
    return np.tile(np.concatenate((wave1, wave2)), frequency)

Let's plot it and observe the rhape of the generated wave.

In [ ]:
@widgets.interact(frequency=(100, 441, 1))
def update(frequency=200):
    
    wavetable = wavetable_alternate(frequency, sine, triangle)

    plt.figure(figsize=figsize)
    
    for i in range(20):
        plt.axvline(x=i*fs/2/frequency, color="orange", linestyle="--")
        plt.axvline(x=i*fs/frequency, color="red", linestyle="-")

    plt.plot(wavetable)
    plt.xlabel("Time [samples]")
    plt.ylabel("Amplitude")
    plt.xlim(0, 1000)
    plt.title("Wavetable alternating between a sine and a triangle wave")
    plt.show()

    plt.figure(figsize=figsize)
    plt.magnitude_spectrum(wavetable, Fs=fs, scale="dB", linewidth=2, alpha=.7, label="Wavetable")
    plt.magnitude_spectrum(triangle(1, frequency*2*np.pi, 0, time[:fs]), Fs=fs, scale="dB", linewidth=2, alpha=.7, label="Triangle")
    plt.magnitude_spectrum(sine(1, frequency*2*np.pi, 0, time[:fs]), Fs=fs, scale="dB", linewidth=2, alpha=.7, label="Sine")

    plt.xlim(0, 1000)
    plt.ylim(-250, 0)
    plt.legend(loc="upper right")
    plt.show()

Let's listen to it!

In [ ]:
Audio(wavetable_alternate(200, sine, triangle), rate=fs)

Similarly, an EG can be set to switch from a harsh sounding to a more mellow waveform as during the decay time. Hence, multiple similar waveforms often are grouped together in order to be switched from and to. 

### 4.2. Morphing

Generally, wavetable synths implement a simple linear interpolation between waveforms, so that the switch from a waveform to another sounds smooth. This is called **morphing**. The below animation shows a simple example of morphing between 2 basic waveforms, square and sine. The user can tune the "morphing amount" to shift the generated waveform closer to the sine or the square wave. 

In [ ]:
# Constants
table_resolution = 101 # Number of samples per wave cycle
table_time = np.linspace(0, 1, table_resolution)

In [ ]:
@widgets.interact(morph=(0, 1, .05), 
                  phase_square=(0, 2*np.pi, .1), 
                  phase_sine=(0, 2*np.pi, .1))
def update(morph=0.5, phase_square=0, phase_sine=0):
    
    wave1 = square(1, 2*np.pi, phase_square, table_time)
    wave2 = sine(1, 2*np.pi, phase_sine, table_time)

    wavetable = np.vstack((wave1, wave2)).T
    morphed_wave = RectBivariateSpline(x=table_time, y=np.arange(wavetable[1].size), z=wavetable, ky=1)

    fig = plt.figure(figsize=(8,8))
    ax = Axes3D(fig) #, auto_add_to_figure=False) # this generates an error in recent versions of matplotlib
    #fig.add_axes(ax) # and this generates a warning 

    x=table_time
    y=table_time
    z=morphed_wave(x, y)

    ax.plot(x, morphed_wave(x, 0), zs=0, zdir='y', linewidth=6, alpha=.7, label='Square wave', zorder=9)
    ax.plot(x, morphed_wave(x, 1), zs=1, zdir='y', linewidth=6, alpha=.3, label='Sine wave', zorder=7)
    ax.plot(x, morphed_wave(x, morph), zs=morph, zdir='y', linewidth=6, alpha=.5, label='Morphed output', zorder=8)

    x, y = np.meshgrid(x, y)
    #dem3d=ax.plot_surface(x.T, y.T, z, cmap=cm.twilight_shifted, linewidth=0, alpha=.7)
    dem3d=ax.plot_wireframe(x.T, y.T, z, cmap=cm.twilight_shifted, alpha=.5, rstride=0, cstride=4, color="grey")
    
    # Turn off tick labels
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_zticklabels([])
    plt.axis('off')

    plt.legend(loc="lower left")
    plt.title("Wavetable morphing between a square and a sine wave")
    #plt.savefig("morph.pdf",bbox_inches='tight')
    plt.show()

The following animation shows the shape of one cycle of the morphed output when the wavetable contains a sine, a triangle, a sawtooth, and a square wave. In orange, we plot the value of the underlying basic waveforms that compose the morphet output!

In [ ]:
@widgets.interact(morph=(0, 4, .1), 
                  phase1=(0, 2*np.pi, .1), 
                  phase2=(0, 2*np.pi, .1), 
                  phase3=(0, 2*np.pi, .1), 
                  phase4=(0, 2*np.pi, .1) )
def update(morph=0, phase1=0, phase2=0, phase3=0, phase4=0):
    
    wave1 = sine(1, 2*np.pi, phase1, table_time)
    wave2 = triangle(1, 2*np.pi, phase2, table_time)
    wave3 = saw(1, 2*np.pi, phase3, table_time)
    wave4 = square(1, 2*np.pi, phase4, table_time)

    wavetable = np.vstack((wave1, wave2, wave3, wave4, wave1)).T
    morphed_wave = RectBivariateSpline(x=table_time, y=np.arange(wavetable[1].size), z=wavetable, ky=1)

    plt.figure(figsize=figsize)
    
    plt.plot(table_time, wave1, linewidth=5, alpha=.7*np.clip(1-np.abs(0-morph),0,1), color="orange")
    plt.plot(table_time, wave2, linewidth=5, alpha=.7*np.clip(1-np.abs(1-morph),0,1), color="orange")
    plt.plot(table_time, wave3, linewidth=5, alpha=.7*np.clip(1-np.abs(2-morph),0,1), color="orange")
    plt.plot(table_time, wave4, linewidth=5, alpha=.7*np.clip(1-np.abs(3-morph),0,1), color="orange")
    plt.plot(table_time, wave1, linewidth=5, alpha=.7*np.clip(1-np.abs(4-morph),0,1), color="orange")
    
    plt.plot(table_time, morphed_wave(table_time, morph), linewidth=5)
    
    plt.axis('off')
    plt.title("Wavetable morphing between sine-tri-saw-square waves")
    plt.show()

Using a LFO, we can automatically change the morphing level over time!

In [ ]:
frequency = 100

wave1 = sine(1, frequency*2*np.pi, 0, time)
wave2 = triangle(1, frequency*2*np.pi, 0, time)
wave3 = saw(1, frequency*2*np.pi, 0, time)
wave4 = square(1, frequency*2*np.pi, 0, time)

wavetable = np.vstack((wave1, wave2, wave3, wave4, wave1)).T
morphed_wave = RectBivariateSpline(x=time, y=np.arange(wavetable[1].size), z=wavetable, ky=1)

morph_LFO = 2 + 2*triangle(1, 2*2*np.pi, 0, time)

wavetable_morph = np.empty(2*fs)

for i in range(2*fs):
    wavetable_morph[i] = morphed_wave(time[i], morph_LFO[i])
    
fig, ax = plt.subplots(2, figsize=(15,4))
ax[1].plot(morph_LFO[:20000], linestyle="--", alpha=.6)
ax[0].plot(wavetable_morph[:20000])
ax[0].set(ylabel='Morphed output')
ax[1].set(ylabel='LFO')
ax[0].set_xticklabels([])
ax[0].set_title("LFO controlled wavetable morphing between sine-tri-saw-square waves")
plt.xlabel("Time [samples]")
plt.show()

Audio(wavetable_morph, rate=fs)